In [1]:
get_ipython().magic(u'matplotlib inline')
%matplotlib tk
# tk needed for animation (could differ on Windows)
import mne
import pandas as pd
import numpy as np
from scipy import stats
from scipy import signal

In [2]:
### Read file ###
path = '/home/akkus/Documents/NumLab/EEG/Megosztott EEG/test1_data/'
raw0 = mne.io.read_raw_egi(path + 'Fanni 20110317 1104002.raw')
raw1 = mne.io.read_raw_egi(path + 'Fanni 20110317 1104004.raw')
# raw.plot(duration=60, n_channels=128, remove_dc=False)

# Set montage
# TODO check if this montage is appropriate for our data
raw0.rename_channels(dict(zip(raw0.ch_names, [ch_name.replace('EG ', '').replace('E0', 'E').replace('E0', 'E') for ch_name in raw0.ch_names])))
raw1.rename_channels(dict(zip(raw1.ch_names, [ch_name.replace('EG ', '').replace('E0', 'E').replace('E0', 'E') for ch_name in raw1.ch_names])))
mont = mne.channels.read_montage(kind='GSN-HydroCel-128')
raw0.set_montage(mont)
raw1.set_montage(mont)
# mne.viz.plot_montage(mont, show_names=1)

Reading EGI header from /home/akkus/Documents/NumLab/EEG/Megosztott EEG/test1_data/Fanni 20110317 1104002.raw...
    Reading events ...
    Assembling measurement info ...
    Synthesizing trigger channel "STI 014" ...
    Excluding events {} ...
Reading EGI header from /home/akkus/Documents/NumLab/EEG/Megosztott EEG/test1_data/Fanni 20110317 1104004.raw...
    Reading events ...
    Assembling measurement info ...
    Synthesizing trigger channel "STI 014" ...
    Excluding events {} ...


In [3]:
### Find events in raw data ###
events0 = mne.find_events(raw0, stim_channel='stim')  # 'STI 014' 'resp' and 'stim' channels include events
events1 = mne.find_events(raw1, stim_channel='stim')
# events = mne.find_events(raw, stim_channel='resp') # for response locked epochs
# Recode events from log file
log_data0 = pd.read_csv(path + 'Fanni 20110317 1104_2.csv', header=None, names=['onset', 'session', 'stim-resp', 'event_type', 'nothing'])
log_data1 = pd.read_csv(path + 'Fanni 20110317 1104_4.csv', header=None, names=['onset', 'session', 'stim-resp', 'event_type', 'nothing'])
# print log_data
events0[:, 2] = log_data0[log_data0['event_type']=='stim']['stim-resp'] # for stim locked epochs
events1[:, 2] = log_data1[log_data1['event_type']=='stim']['stim-resp']
# events[:, 2] = log_data[log_data['event_type']=='resp']['stim-resp'].replace(to_replace=['p', 'q'], value=[1, 2])  # for response locked epochs
#print events
# mne.viz.plot_events(events)
# mne.viz.plot_events(events2)

320 events found
Events id: [1]
320 events found
Events id: [1]


In [4]:
# Deleting the first two and the last events from each block (to give relatively clean data to the ICA algorithm)
raw0.crop(tmin=(events0[2][0]-200)/1000, tmax=(events0[len(events0)-1][0])/1000, copy=False)
raw1.crop(tmin=(events1[2][0]-200)/1000, tmax=(events1[len(events1)-1][0])/1000, copy=False)

events0 = np.delete(events0, [0,1,len(events0)-1], 0)
events1 = np.delete(events1, [0,1,len(events1)-1], 0)

In [5]:
# Concatenating raw files
raw, events = mne.concatenate_raws([raw0, raw1], events_list=[events0, events1])
del raw0, raw1, events0, events1, log_data0, log_data1
# TODO Maybe the raw files should be handled separately until Stage III is completed (ICA)? This possibly depends on whether an adjustment was made by hand between blocks.

In [6]:
### Setting Fz as reference ###
raw.load_data()
fz_ch = ['E11'] # According to the offical topomap, E11 is the Fz electrode
raw, _ = mne.io.set_eeg_reference(raw, fz_ch)

Reading 0 ... 883001  =      0.000 ...   883.001 secs...
Applying a custom EEG reference.


In [7]:
### Filtering ###
# picks = mne.pick_types(raw_faster.info, meg=False, eeg=True, eog=False, stim=False, exclude='bads')
# raw.plot_psd(area_mode='range', tmax=10.0, fmax=50, picks=picks) # Plot the raw frequency power before filtering

high_pass = 4.0
low_pass = 30.0
raw.filter(high_pass, low_pass)  # high pass over 4 Hz and low pass below 30 Hz
# Plot the filtered frequency power
# raw.plot_psd(area_mode='range', tmax=10.0, fmax=100, picks=picks)

Band-pass filtering from 4 - 30 Hz


<RawEGI  |  Fanni 20110317 1104002.raw, n_channels x n_times : 131 x 883002 (883.0 sec)>

In [8]:
### Defining channel types ###
stim_chs = ['STI 014', 'resp', 'stim']
eog_chs = ['E17', 'E48', 'E119', 'E125', 'E126', 'E127', 'E128'] # TODO These are definitely not EEG channels, however some of them may not be EOG, but EMG (maybe ECG?) channel

for ch in eog_chs:
    raw.set_channel_types({ch:'eog'})

# Channels to include in the analysis (using EEG channels only)
eeg_chs = [ch for ch in raw.info['ch_names'] if ch not in stim_chs + eog_chs + fz_ch]

In [9]:
### Hurst exponent function from 'eegfaster' ###
# TODO Check if this is really an accurate implementation
#KA nem érhető el ez már Pythonban? pl https://pypi.python.org/pypi/nolds vagy 
#https://pypi.python.org/pypi/heteromotility/
#KA Octave-val lehet ellenőrizni, hogy ugyanat csinálják-e
def hurst(x):
    """FASTER [Nolan2010]_ implementation of the Hurst Exponent.
    Parameters
    ----------
    x : numpy.ndarray
        Vector with the data sequence.
    Returns
    -------
    h : float
        Computed hurst exponent
    #-SPHINX-IGNORE-#
    References
    ----------
    [Nolan2010] H. Nolan, R. Whelan, and R.B. Reilly. Faster: Fully automated
    statistical thresholding for eeg artifact rejection. Journal of
    Neuroscience Methods, 192(1):152-162, 2010.
    #-SPHINX-IGNORE-#
    """

    # Get a copy of the data
    x0 = x.copy()
    x0_len = len(x)

    yvals = np.zeros(x0_len)
    xvals = np.zeros(x0_len)
    x1 = np.zeros(x0_len)

    index = 0
    binsize = 1

    while x0_len > 4:

        y = x0.std()
        index += 1
        xvals[index] = binsize
        yvals[index] = binsize*y

        x0_len /= 2
        binsize *= 2
        for ipoints in xrange(x0_len):
            x1[ipoints] = (x0[2*ipoints] + x0[2*ipoints - 1])*0.5

        x0 = x1[:x0_len]

    # First value is always 0
    xvals = xvals[1:index+1]
    yvals = yvals[1:index+1]

    logx = np.log(xvals)
    logy = np.log(yvals)

    p2 = np.polyfit(logx, logy, 1)
    return p2[0]

In [70]:
### FASTER ###
from multiprocessing import Pool

# Arguments
crit_z = 3
tmin = -0.5  # start of each epoch (500ms before the trigger)
tmax = 1.5  # end of each epoch (1500ms after the trigger)
baseline = (-0.2, 0)  # means from the first instant to t = - 200ms (as suggested by Nolan et al. (2010))
event_id = [1,2,3,4,6,7,8,9]
picks_epochs = mne.pick_types(raw.info, meg=False, eeg=True, eog=True, stim=False, exclude=[])

# Functions for single parameters (multiprocessing.Pool.map() needs top level functions which are pickleable)
def get_channels_corr(ch_pair, data=None):
    if not data: data = df
    ch_x = ch_pair[0]
    ch_y = ch_pair[1]
    chs_corr = stats.pearsonr(data[ch_x], data[ch_y])[0]
    return (ch_pair, chs_corr)

# def get_channel_mcorr(ch, data=None): # TODO This function is extremely slow! 
#     if not data: data = df
#     ch_mcorr = np.mean([stats.pearsonr(data[ch], data[ch2])[0] for ch2 in eeg_chs if ch2 != ch])
#     return ch_mcorr                       

def get_channel_mcorr(ch, c_dict=None):
    if not c_dict: c_dict = corr_dict
    ch_mcorr = np.mean([c_dict[key] for key in c_dict.keys() if ch in key])
    return ch_mcorr

def get_channel_var(ch, data=None):
    if not data: data = df
    ch_var = np.var(data[ch])
    return ch_var
                         
def get_channel_hurst(ch, data=None):
    if not data: data = df
    ch_hurst = hurst(np.asarray(data[ch]))
    return ch_hurst

def get_channel_ampl(ch, data=None):
    if not data: data = df
    ch_ampl = max(data[ch]) - min(data[ch])
    return ch_ampl

def get_channel_dev(ch, data=None, mean=None):
    if not data: data = df
    if not mean: mean = ch_means[ch]
    ch_dev = mean - np.mean(data[ch])
    return ch_dev

# Defining parameter lists for stages
params_bad_channels = [get_channel_mcorr, get_channel_var, get_channel_hurst] # Parameters 1, 2, & 3 [Stage I.] (Nolan et al., 2010)
params_bad_epochs = [get_channel_ampl, get_channel_dev, get_channel_var] # Parameters 4, 5, & 6 [Stage II.] (Nolan et al., 2010)


# # Finding bad channels (Stage I.)
# def find_bad_channels(raw, eeg_chs=eeg_chs, crit_z=crit_z):
#     # Initializing Pool for multiprocessing
#     def initializer(): # Setting up global variables for the child processes
#         global df
#         df = raw.to_data_frame()
#     pool = Pool(None, initializer, ())
#     # Computing parameters
#     bads = []
#     for param in params_bad_channels:
#         z_scores = stats.zscore(pool.map(param, eeg_chs))
#         # Defining channel outliers
#         bads = bads + [eeg_chs[i] for i, z in enumerate(z_scores) if abs(z) > crit_z and eeg_chs[i] not in bads]
#     # Closing Pool
#     pool.close()
#     pool.join()
    
#     return bads

# Finding bad channels (Stage I.)
def find_bad_channels(raw, eeg_chs=eeg_chs, crit_z=crit_z):
    def get_corr_dict(raw, eeg_chs=eeg_chs):
        def get_channel_pairs(eeg_chs=eeg_chs):
            ch_pairs = []
            for ch_x in eeg_chs:
                for ch_y in eeg_chs:
                    if ch_x != ch_y and (ch_y, ch_x) not in ch_pairs:
                        ch_pairs.append((ch_x, ch_y))
            return ch_pairs
        ch_pairs = get_channel_pairs()
        df_glob = raw.to_data_frame()
        def initializer(): # Setting up global variables for the child processes
            global df
            df = df_glob
        pool = Pool(None, initializer, ())
        # Get channel correlations
        corr_dict = dict(pool.map(get_channels_corr, ch_pairs))
        # Closing Pool
        pool.close()
        pool.join()
        return corr_dict, df_glob
    # Get correlation table for Parameter 1
    corr_dict_glob, df_glob = get_corr_dict(raw)
    # Initializing Pool for multiprocessing
    def initializer(): # Setting up global variables for the child processes
        global df, corr_table
        df = df_glob
        corr_dict = corr_dict_glob
    pool = Pool(None, initializer, ())
    # Computing parameters
    bads = []
    for param in params_bad_channels:
        z_scores = stats.zscore(pool.map(param, eeg_chs))
        # Defining channel outliers
        bads = bads + [eeg_chs[i] for i, z in enumerate(z_scores) if abs(z) > crit_z and eeg_chs[i] not in bads]
    # Closing Pool
    pool.close()
    pool.join()
    return bads

# Finding bad epochs (Stage II.)
def find_bad_epochs(raw, events=events, event_id=event_id, tmin=tmin, tmax=tmax, picks=picks_epochs, baseline=baseline):
    # Get epochs data
    epochs_df = mne.Epochs(raw, events, event_id, tmin, tmax, picks=picks, baseline=baseline, preload=False, proj=True).to_data_frame()
    # Get channel means across epochs for Parameter 5
    ch_means_glob = pd.DataFrame([np.mean(epochs_df[ch]) for ch in eeg_chs], index=eeg_chs).transpose()
    # Iterating over epochs
    epochs_vals = np.empty((0,3))
    for epoch in set(epochs_df.index.get_level_values(level='epoch')):
        # Initializing Pool for multiprocessing
        def initializer(): # Setting up global variables for the child processes
            global df, ch_means
            df = epochs_df.xs(epoch, level='epoch')
            ch_means = ch_means_glob # TODO 'ch_means = ch_means' returns with an error
        pool = Pool(None, initializer, ())
        # Computing parameters for each epoch
        vals = []
        for param in params_bad_epochs:        
            vals.append(np.mean(pool.map(param, eeg_chs)))
        epochs_vals = np.vstack([epochs_vals, np.asarray([vals])])
        # Closing Pool   
        pool.close()
        pool.join()
    # Defining epoch outliers   
    z_scores = stats.zscore(epochs_vals)        
    bads = [i for i, z in enumerate(z_scores) if abs(z[0]) > crit_z or abs(z[1]) > crit_z or abs(z[2]) > crit_z]
    return bads

In [71]:
%%time
bad_channels = find_bad_channels(raw)

Converting time column to int64...
CPU times: user 1.9 s, sys: 676 ms, total: 2.57 s
Wall time: 2min 29s


In [72]:
print bad_channels
n_interpolated = len(bad_channels)
raw.info['bads'] = bad_channels
raw.interpolate_bads(reset_bads=True)

['E8', 'E14', 'E21', 'E25', 'E9']
Computing interpolation matrix from 116 sensor positions
Interpolating 5 sensors


<RawEGI  |  Fanni 20110317 1104002.raw, n_channels x n_times : 131 x 883002 (883.0 sec)>

In [13]:
%%time
bad_epochs = find_bad_epochs(raw)

634 matching events found
Applying baseline correction (mode: mean)
0 projection items activated
Loading data for 634 events and 2001 original time points ...
1 bad epochs dropped
Converting time column to int64...
CPU times: user 11.9 s, sys: 37.6 s, total: 49.5 s
Wall time: 2min 3s


In [14]:
print bad_epochs
epochs = mne.Epochs(raw, events, event_id, tmin, tmax, picks=picks_epochs, baseline=baseline, preload=False, proj=True)
epochs.drop(bad_epochs)

[174, 188, 271, 372, 527, 531, 532, 631]
634 matching events found
Applying baseline correction (mode: mean)
0 projection items activated
Dropped 8 epochs


<Epochs  |  n_events : 626 (good & bad), tmin : -0.5 (s), tmax : 1.5 (s), baseline : (-0.2, 0),
 '1': 79, '2': 79, '3': 80, '4': 80, '6': 77, '7': 76, '8': 75, '9': 80>